# Загрузка данных + импорты

## Импорты

In [307]:
#импорты
#from nltk.tokenize import word_tokenize, RegexpTokenizer
#from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, cross_validate
import pandas as pd
from sklearn.naive_bayes import ComplementNB
#from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from catboost import CatBoostClassifier
from sklearn.model_selection import KFold
from scipy.stats import ttest_ind
from scipy import stats as st
import numpy as np

In [235]:
FILESOURCE = 'SMSSpamCollection.txt' # файл источник


## Чтение файла

In [236]:
# читаем файл
with open(FILESOURCE) as f: 
    full_file = f.read()
print(full_file[:20])

ham	Go until jurong 


**Вывод:** данные удалось прочитать, уже неплохо

# Предобработка

In [237]:
# нижний регистр и разделение на строки.
full_file = full_file.lower()
lst = full_file.splitlines()

In [238]:
# Отделение таргета
lst_with_target = []
for line in lst:
    lst_tmp = []
    lst_line = line.split('\t')  
    if lst_line[0] == 'spam' or lst_line[0] == 'ham':
        lst_tmp.append('ok')
        lst_tmp.append(lst_line[0])
        lst_tmp.append(lst_line[1:])
        lst_with_target.append(lst_tmp)
    else:
        lst_tmp.append('not_ok')
        lst_tmp.append(lst_line[0])
        lst_tmp.append(lst_line[1:])
        lst_with_target.append(lst_tmp)

In [239]:
# проверка на структуру файла
for i in range(len(lst_with_target)):
    if lst_with_target[i][0] != 'ok':
        print(lst_with_target[i], 'is not ok')
    if len(lst_with_target[i]) > 3:
        print(lst_with_target[i], '>3')

**Вывод:** структура файла соблюдается

In [240]:
vectorizer = CountVectorizer(stop_words='english', token_pattern='[a-z]\w+')
X = vectorizer.fit_transform(lst)
#vectorizer.get_feature_names_out()


In [241]:
vectorizer2 = CountVectorizer(token_pattern='[0-9]\w+')
X2 = vectorizer2.fit_transform(lst)
vectorizer3 = CountVectorizer(token_pattern='[§!@#$%^&*()_+-<>:"|\?~{}]')
X3 = vectorizer3.fit_transform(lst)


In [242]:
df = pd.DataFrame(X.toarray(), columns= vectorizer.get_feature_names_out())
df2 = pd.DataFrame(X2.toarray(), columns= vectorizer2.get_feature_names_out())
df3 = pd.DataFrame(X3.toarray(),columns=vectorizer3.get_feature_names_out())

In [243]:
df['words_with_digits'] = df2.T.sum()
df['special_symbols'] = df3.T.sum()

In [244]:
df['target'] =  (df['spam'] > 0).apply(int)

df = df.drop(['spam', 'ham'], axis=1)

In [245]:
len(vectorizer.get_feature_names_out())

7680

In [246]:
# Создаем транспонированый df
df_t = df.T

## Среднее количество слов в сообщении

In [247]:
df[df['target'] == 1].T.sum().mean()

39.163319946452475

In [248]:
df[df['target'] == 0].T.sum().mean()

10.946550652579242

In [249]:
# разделение
features = df.drop('target',axis=1)
target = df['target']
features_train, features_test, target_train, target_test = train_test_split(features,target, test_size=.3, random_state=123456)
print(features_train.shape, features_test.shape, target_train.shape, target_test.shape )

(3901, 7680) (1673, 7680) (3901,) (1673,)


In [250]:
features_train['len_diff'] = abs(df_t.sum() - features_train[target_train == 1].T.sum().mean())
features_test['len_diff'] = abs(df_t.sum() - features_train[target_train == 1].T.sum().mean())
features['len_diff'] = abs(df_t.sum() - features_train[target_train == 1].T.sum().mean())

### Сбалансированный df

In [251]:
features_train_ones = features_train[target_train == 1]

In [252]:
features_train_zeros = features_train[target_train == 0].sample(n=features_train_ones.shape[0])

In [253]:
features_train_balanced = pd.concat([features_train_ones, features_train_zeros])

In [254]:
features_train_balanced = features_train_balanced.join(target_train)
features_train_balanced = shuffle(features_train_balanced)

In [255]:
target_train_balanced = features_train_balanced['target']

In [256]:
features_train_balanced = features_train_balanced.drop('target', axis=1)

# EDA

## Баланс классов

### Общий баланс классов

In [257]:
df['target'].mean()

0.1340150699677072

### Баланс классов train

In [258]:
target_train.mean()

0.12971033068443988

In [259]:
target_train_balanced.mean()

0.5

### Баланс классов test

In [260]:
target_test.mean()

0.14405260011954574

## Поиск важных слов


In [261]:
df[df['target'] == 1].sum().sort_values(ascending=False)[:25]

special_symbols      15925
words_with_digits     2540
target                 747
free                   226
txt                    166
ur                     144
mobile                 128
stop                   125
text                   125
claim                  113
reply                  104
www                     98
prize                   93
just                    79
cash                    76
won                     76
uk                      74
send                    71
new                     69
nokia                   67
win                     64
urgent                  63
week                    60
tone                    60
service                 56
dtype: int64

In [262]:
df[df['target'] == 0].sum().sort_values(ascending=False)[:25]

special_symbols      18803
words_with_digits      356
gt                     318
lt                     316
just                   297
ok                     288
ll                     266
ur                     247
know                   237
good                   235
got                    234
like                   234
come                   230
day                    217
love                   205
time                   202
going                  169
home                   165
want                   165
lor                    162
don                    159
need                   159
sorry                  157
da                     150
today                  139
dtype: int64

**Вывод:** Можно утверждать, что в spam-сообщениях часто употребляются такие слова как free и txt, а также есть много слов с цифрами

## Модели

In [263]:
cnb = ComplementNB()
cnb_bal = ComplementNB()
lgr = LogisticRegression(max_iter=2000)
cbc = CatBoostClassifier(verbose=0)

In [264]:
scoring = ['f1']
def make_cv2(model_name, model_, cv, scoring,features_train, target_train):
    scores = cross_validate(model_, features_train, target_train, cv=cv, scoring=scoring)
    print(model_name)
    print(f'mean=',scores['test_f1'].mean())
    print(f'std=',scores['test_f1'].std())

In [265]:
make_cv2('NB not balanced', cnb, 5, scoring, features_train, target_train)
make_cv2('NB balanced', cnb_bal, 5, scoring, features_train_balanced, target_train_balanced)
make_cv2('log_reg_balanced', lgr, 5, scoring, features_train_balanced, target_train_balanced)
make_cv2('catboost', cbc, 5, scoring, features_train_balanced, target_train_balanced)


NB not balanced
mean= 0.8479433295125964
std= 0.038820387690466834
NB balanced
mean= 0.9405555840520545
std= 0.02080961306435698
log_reg_balanced
mean= 0.9659671262224336
std= 0.009829848014354842
catboost
mean= 0.9553725768770907
std= 0.00924145948723892


In [266]:
cnb.fit(features_train,target_train)
cnb_bal.fit(features_train_balanced, target_train_balanced)
lgr.fit(features_train_balanced, target_train_balanced)
cbc.fit(features_train_balanced, target_train_balanced, verbose=0)

In [267]:
def print_model_rep(model_kind, model, features_train, target_train, features_test, target_test):
    print(model_kind)
    print('train')
    prediction = model.predict(features_train)
    print(classification_report(target_train, prediction))
    print('test')
    prediction = model.predict(features_test)
    print(classification_report(target_test, prediction))



In [268]:
print_model_rep('NB not balanced', cnb, features_train, target_train, features_test, target_test)

NB not balanced
train
              precision    recall  f1-score   support

           0       0.99      0.97      0.98      3395
           1       0.84      0.93      0.88       506

    accuracy                           0.97      3901
   macro avg       0.91      0.95      0.93      3901
weighted avg       0.97      0.97      0.97      3901

test
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      1432
           1       0.93      0.86      0.89       241

    accuracy                           0.97      1673
   macro avg       0.95      0.93      0.94      1673
weighted avg       0.97      0.97      0.97      1673



In [269]:
print_model_rep('NB balanced', cnb_bal, features_train_balanced, target_train_balanced, features_test, target_test)

NB balanced
train
              precision    recall  f1-score   support

           0       0.94      0.97      0.96       506
           1       0.97      0.94      0.96       506

    accuracy                           0.96      1012
   macro avg       0.96      0.96      0.96      1012
weighted avg       0.96      0.96      0.96      1012

test
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      1432
           1       0.92      0.86      0.89       241

    accuracy                           0.97      1673
   macro avg       0.95      0.92      0.94      1673
weighted avg       0.97      0.97      0.97      1673



In [270]:
print_model_rep('log_reg_balanced', lgr, features_train_balanced, target_train_balanced, features_test, target_test)


log_reg_balanced
train
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       506
           1       1.00      0.99      1.00       506

    accuracy                           1.00      1012
   macro avg       1.00      1.00      1.00      1012
weighted avg       1.00      1.00      1.00      1012

test
              precision    recall  f1-score   support

           0       0.98      0.99      0.99      1432
           1       0.96      0.90      0.93       241

    accuracy                           0.98      1673
   macro avg       0.97      0.95      0.96      1673
weighted avg       0.98      0.98      0.98      1673



In [271]:
print_model_rep('catboost', cbc, features_train_balanced, target_train_balanced, features_test, target_test)


catboost
train
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       506
           1       1.00      1.00      1.00       506

    accuracy                           1.00      1012
   macro avg       1.00      1.00      1.00      1012
weighted avg       1.00      1.00      1.00      1012

test
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      1432
           1       0.88      0.93      0.90       241

    accuracy                           0.97      1673
   macro avg       0.94      0.95      0.94      1673
weighted avg       0.97      0.97      0.97      1673



## Кросс-валидация

In [272]:
kf = KFold(n_splits=5)
kf.get_n_splits()

5

In [273]:
def get_cv_values(model_kind, model, features, target, kf ):
    num_fold = 0
    for train_index, test_index in kf.split(features):
        num_fold += 1
        print(f'Fold number {num_fold}')
        print_model_rep(model_kind, model, features.loc[train_index], target[train_index], features.loc[test_index], target[test_index])

In [274]:
get_cv_values('NB not balanced', cnb, features, target, kf)

Fold number 1
NB not balanced
train
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      3880
           1       0.89      0.88      0.88       579

    accuracy                           0.97      4459
   macro avg       0.93      0.93      0.93      4459
weighted avg       0.97      0.97      0.97      4459

test
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       947
           1       0.94      0.90      0.92       168

    accuracy                           0.98      1115
   macro avg       0.96      0.94      0.95      1115
weighted avg       0.98      0.98      0.98      1115

Fold number 2
NB not balanced
train
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      3853
           1       0.91      0.88      0.89       606

    accuracy                           0.97      4459
   macro avg       0.94      0.93      0.94      44

In [275]:
get_cv_values('NB balanced', cnb_bal, features, target, kf)

Fold number 1
NB balanced
train
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      3880
           1       0.88      0.88      0.88       579

    accuracy                           0.97      4459
   macro avg       0.93      0.93      0.93      4459
weighted avg       0.97      0.97      0.97      4459

test
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       947
           1       0.94      0.89      0.92       168

    accuracy                           0.98      1115
   macro avg       0.96      0.94      0.95      1115
weighted avg       0.98      0.98      0.98      1115

Fold number 2
NB balanced
train
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      3853
           1       0.90      0.88      0.89       606

    accuracy                           0.97      4459
   macro avg       0.94      0.93      0.94      4459
weigh

In [276]:
get_cv_values('log_reg_balanced', lgr, features, target, kf)



Fold number 1
log_reg_balanced
train
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      3880
           1       0.95      0.93      0.94       579

    accuracy                           0.98      4459
   macro avg       0.97      0.96      0.97      4459
weighted avg       0.98      0.98      0.98      4459

test
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       947
           1       0.97      0.94      0.95       168

    accuracy                           0.99      1115
   macro avg       0.98      0.97      0.97      1115
weighted avg       0.99      0.99      0.99      1115

Fold number 2
log_reg_balanced
train
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      3853
           1       0.96      0.93      0.94       606

    accuracy                           0.98      4459
   macro avg       0.97      0.96      0.97      

In [277]:
get_cv_values('catboost', cbc, features, target, kf)

Fold number 1
catboost
train
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      3880
           1       0.86      0.94      0.90       579

    accuracy                           0.97      4459
   macro avg       0.93      0.96      0.94      4459
weighted avg       0.97      0.97      0.97      4459

test
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       947
           1       0.92      0.95      0.94       168

    accuracy                           0.98      1115
   macro avg       0.96      0.97      0.96      1115
weighted avg       0.98      0.98      0.98      1115

Fold number 2
catboost
train
              precision    recall  f1-score   support

           0       0.99      0.98      0.99      3853
           1       0.89      0.94      0.91       606

    accuracy                           0.98      4459
   macro avg       0.94      0.96      0.95      4459
weighted av

## Гипотеза о равенстве средних

In [284]:
samples = []
for _ in range(50):
    samples.append(features.sample(n=100,replace=True))

In [285]:
def get_f1(model, samples):
    f1_lst = []
    for elem in samples:
        elem_features = elem
        elem_target = target[elem_features.index]
        prediction = model.predict(elem_features)
        f1_lst.append(f1_score(elem_target, prediction))
    return f1_lst


In [334]:
def conf_int(f1_lst):
    f1_lst = pd.Series(f1_lst)
    print(f1_lst.mean())
    mean_f1 = f1_lst.mean()
    t_low = f1_lst.quantile(.005)
    t_high = f1_lst.quantile(.995)
    sem_mean = f1_lst.sem()
    return mean_f1 + t_low * sem_mean, mean_f1 + t_high * sem_mean
    

## Доверительный интервал

In [335]:
conf_int(get_f1(cnb, samples))

0.8910154542274433


(0.8977830324341752, 0.9012541443371611)

In [336]:
conf_int(get_f1(cnb_bal, samples))

0.8904058136425063


(0.8974058285469362, 0.9004529503894804)

In [337]:
conf_int(get_f1(lgr, samples))

0.9446564774860431


(0.9506309098578992, 0.9520370227768512)

In [338]:
conf_int(get_f1(cbc, samples))

0.904904041462919


(0.9111343904195075, 0.9135035569309501)

In [327]:
print(ttest_ind(get_f1(cnb, samples),get_f1(cnb_bal, samples),equal_var=False))

Ttest_indResult(statistic=0.042498812335333994, pvalue=0.9661875582755408)


In [287]:
print(ttest_ind(get_f1(cnb_bal, samples),get_f1(lgr, samples),equal_var=False))

Ttest_indResult(statistic=-4.3516627379775175, pvalue=3.561377093917459e-05)


In [288]:
print(ttest_ind(get_f1(cnb_bal, samples),get_f1(cbc, samples),equal_var=False))

Ttest_indResult(statistic=-1.0962868574893463, pvalue=0.2757037732272059)


**Вывод** нам удалось сделать модель, которая в некоторой степени умеет классифицировать сообщения на SPAM и HAM